In [22]:
import csv
from faker import Faker
from datetime import datetime, timedelta
import random

In [23]:
fake = Faker()

authors_id = -1
books_id = -1
address_id = -1
customers_id = -1
orders_id = -1
order_items_id = -1

tables = ['authors', 'books', 'addresses', 'customers', 'orders', 'order_items']

In [24]:
def generate_authors():
    global authors_id
    with open('csv/authors.csv', 'w', newline='') as csvfile:
        fieldnames = ['id', 'first_name', 'last_name', 'bio']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for i in range(100):
            authors_id += 1
            writer.writerow({'id': authors_id, 'first_name': fake.first_name(), 'last_name': fake.last_name(), 'bio': fake.text()})


In [25]:
def generate_books():
    global books_id
    with open('csv/books.csv', 'w', newline='') as csvfile:
        fieldnames = ['id', 'title', 'genres', 'published_at', 'price', 'stock', 'author_id']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for i in range(1000):
            books_id += 1
            writer.writerow({'id': books_id, 'title': fake.sentence(), 'genres': fake.text(), 'published_at': fake.date_time_this_decade(), 'price': random.uniform(1, 100), 'stock': random.randint(1, 100), 'author_id': random.randint(0, authors_id)})

In [26]:
def generate_address():
    global address_id
    with open('csv/addresses.csv', 'w', newline='') as csvfile:
        fieldnames = ['id', 'street', 'city', 'state', 'postal_code', 'country']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for i in range(100):
            address_id += 1
            writer.writerow({'id': address_id, 'street': fake.street_address(), 'city': fake.city(), 'state': fake.state(), 'postal_code': fake.postcode(), 'country': fake.country()})

In [27]:
def generate_customers():
    global customers_id
    with open('csv/customers.csv', 'w', newline='') as csvfile:
        fieldnames = ['id', 'name', 'email', 'created_at', 'address_id']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for i in range(100):
            customers_id += 1
            writer.writerow({'id': customers_id, 'name': fake.name(), 'email': fake.email(), 'created_at': fake.date_time_this_decade(), 'address_id': random.randint(0, address_id)})

In [28]:
def generate_orders():
    global orders_id
    with open('csv/orders.csv', 'w', newline='') as csvfile:
        fieldnames = ['id', 'customer_id', 'total_price', 'created_at', 'status']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for i in range(100):
            orders_id += 1
            writer.writerow({'id': orders_id, 'customer_id': random.randint(0, customers_id), 'total_price': random.uniform(1, 100), 'created_at': fake.date_time_this_decade(), 'status': fake.word()})

In [29]:
def generate_order_items():
    global order_items_id
    with open('csv/order_items.csv', 'w', newline='') as csvfile:
        fieldnames = ['id', 'order_id', 'book_id', 'quantity']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for i in range(100):
            order_items_id += 1
            writer.writerow({'id': order_items_id, 'order_id': random.randint(0, orders_id), 'book_id': random.randint(0, books_id), 'quantity': random.randint(1, 100)})

In [30]:
generate_authors()
generate_books()
generate_address()
generate_customers()
generate_orders()
generate_order_items()

In [31]:
def csv_to_sql_insert(csv_file):
    table_name = csv_file.replace(".csv", "")

    with open("csv/" + csv_file, 'r', newline='', encoding='latin1') as csvfile:
        csv_reader = csv.DictReader(csvfile)
        columns = csv_reader.fieldnames
        
        x = str(tuple(columns)).replace("'", "") 
        insert_queries = []
        
        for row in csv_reader:
            values = [f"{row[column]}" if isinstance(row[column], str) else str(row[column]) for column in columns]
            y = str(tuple(values)).replace("'NULL'", "NULL")
            insert_query = f'INSERT INTO {table_name} {x} VALUES {y};'
            insert_queries.append(insert_query)

    return insert_queries 


insert_queries = []

for tablename in tables:
    insert_queries += csv_to_sql_insert(f"{tablename}.csv")


In [32]:
Syncronisator = """
CREATE OR REPLACE PROCEDURE sync_serial_sequence(table_name TEXT, column_name TEXT)
LANGUAGE plpgsql
AS $$
DECLARE
    sequence_name TEXT;
    max_value BIGINT;
BEGIN
    SELECT pg_get_serial_sequence(table_name, column_name) INTO sequence_name;

    IF sequence_name IS NULL THEN
        RAISE EXCEPTION 'No sequence found for table "%" and column "%"', table_name, column_name;
    END IF;

    EXECUTE FORMAT('SELECT COALESCE(MAX(%I), 0) FROM %I', column_name, table_name) INTO max_value;
    EXECUTE FORMAT('SELECT SETVAL(%L, %s)', sequence_name, max_value + 1);
    RAISE NOTICE 'Sequence "%" synchronized to %', sequence_name, max_value + 1;
END;
$$;

CALL sync_serial_sequence('authors', 'id');
CALL sync_serial_sequence('books', 'id');
CALL sync_serial_sequence('addresses', 'id');
CALL sync_serial_sequence('customers', 'id');
CALL sync_serial_sequence('orders', 'id');
CALL sync_serial_sequence('order_items', 'id');
"""

with open("../../sql/database-dummyloader.sql", "w+", encoding='utf-8') as file:
    for query in insert_queries:
        file.write(query + "\n")

    file.write(Syncronisator)